In [41]:
# imports
# system related
import sys
import subprocess

# basics
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import datetime

# snap7 and logging
import snap7 as s7
import csv



print('imports succesful')

imports succesful


## Connecting to plc

In [42]:
# connecting to plc

plc = s7.client.Client()
plc.connect('192.168.0.19', 0, 1, )  # ip, rack, slot

print(f'connection status: {plc.get_connected()}')

connection status: True


## Defining get & set functions for plc variables

In [43]:
# Defining read/write functions

def get_bool(db_number, start_offset, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    value = s7.util.get_bool(data, 0, bit_offset)
    # print(f'db number: {db_number}, bit: {start_offset}, has boolean value: {value}')
    return value
    
def set_bool(db_number, start_offset, value, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    s7.util.set_bool(data, 0, bit_offset, value)
    plc.db_write(db_number, start_offset, data)
    return None


def get_real(db_number, start_offset):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    real = s7.util.get_real(data, 0)
    return real

def set_real(db_number, start_offset, value):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    s7.util.set_real(data, 0, value)
    plc.db_write(db_number, start_offset, data)
    return None



    


# start make datalog function for ixon db

In [51]:
# ixon_remake.




def processing_loop(csvfile):
    # whole_start = datetime.datetime.now()
    
    # defining constants
    db_log = 33

    log_so_width  = 0
    log_so_rpm    = 4
    log_so_rpm_sp = 8
    log_so_torque = 12
    log_so_hz1    = 16
    log_so_hz1_sp = 20
    log_so_hz2    = 24
    log_so_hz2_sp = 28
    log_so_hz3    = 32
    log_so_hz3_sp = 36
    log_so_hz4    = 40
    log_so_hz4_sp = 44
    
    cols = ['width', 'rpm', 'rpm_sp', 'torque', 'hz1', 'hz1_sp', 'hz2', 'hz2_sp', 'hz3', 'hz3_sp', 'hz4', 'hz4_sp']

    dt = datetime.timedelta(milliseconds=50)
    new_log_time = datetime.datetime.now()
    new_log_time = new_log_time.replace(second = new_log_time.second + 1, microsecond = 0)
    
    # write headers to the .csv file
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['time'] + cols)


    while True:
        # start_time = datetime.datetime.now()

        # waiting untill enough time has elapst
        new_log_time = new_log_time + dt
        
        # print(f'                                          log time is: {new_log_time}')
        # print(f'wait time is {new_log_time  - datetime.datetime.now()}')
        
        while datetime.datetime.now() < new_log_time:
            pass
        # read whole db
        data =  plc.db_read(db_log, 0, 48)

        ######## KAN DIT vvvvvvvvvvv MISSCHIEN VEEL SNELLER???
        vals = [new_log_time]
        for i, name in enumerate( cols):
            val = s7.util.get_real(data, 4*i)
            vals.append(val)
        #     print(f'{name} = {val} ')
        # print('\n')

        ######### DIT ^^^^^^^^
        
        csv_writer.writerow(vals)
        csvfile.flush()

        # end_time = datetime.datetime.now()
        # loop_dt = end_time - start_time
        # print('loop_dt is: ', loop_dt)
    
# def processing_loop(csvfile):
#     csv_writer = csv.writer(csvfile)
#     csv_writer.writerow(['time'] + cols)

#     while True:
#         csv_writer.writerow([new_log_time] + vals)
#         csvfile.flush()

with open('results.csv', 'w') as csvfile:
    processing_loop(csvfile)


        
    
# whole_end = datetime.datetime.now()

# print(f'________________________\n\nMean time is: {(whole_end-whole_start)/10}')




KeyboardInterrupt: 

### another way of unpacking a database data (Thijs)

In [ ]:
import struct
# struct.unpack("b",plc.db_read(db_number, 2, 1)) 